### Data Preparation and Cleaning



<br>

**Import necessary libraries**

In [4]:
import numpy as np
import pandas as pd

<br>

**Import data from csv file**

In [5]:
michelindata = pd.read_csv('michelin_my_maps.csv')
michelindata.head()

,Name,Address,Location,MinPrice,MaxPrice,Currency,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award
0,Aqua,"Parkstraße 1, Wolfsburg, 38440, Germany",Wolfsburg,225,225,EUR,"Creative, Modern Cuisine",10.789999,52.433172,4.953616e+11,https://guide.michelin.com/en/niedersachsen/wo...,http://www.restaurant-aqua.com,3 MICHELIN Stars
1,The Table Kevin Fehling,"Shanghaiallee 15, Hamburg, 20457, Germany",Hamburg,230,230,EUR,Creative,10.002980,53.542623,4.940229e+11,https://guide.michelin.com/en/hamburg-region/h...,http://www.the-table-hamburg.de/,3 MICHELIN Stars
2,Restaurant Überfahrt Christian Jürgens,"Überfahrtstraße 10, Rottach-Egern, 83700, Germany",Rottach-Egern,259,319,EUR,Creative,11.758229,47.696685,4.980227e+09,https://guide.michelin.com/en/bayern/rottach-e...,http://www.althoffcollection.com,3 MICHELIN Stars
3,Victor's Fine Dining by christian bau,"Schlossstraße 27, Perl, 66706, Germany",Perl,205,295,EUR,Creative,6.387211,49.535173,4.968668e+10,https://guide.michelin.com/en/saarland/perl/re...,https://www.victors-fine-dining.de/,3 MICHELIN Stars
4,Rutz,"Chausseestraße 8, Berlin, 10115, Germany",Berlin,198,245,EUR,"Modern Cuisine, Creative",13.386087,52.528351,4.930246e+11,https://guide.michelin.com/en/berlin-region/be...,https://www.rutz-restaurant.de/,3 MICHELIN Stars


The dataset has 13 columns of data; Name, Address, Location, MinPrice, MaxPrice, Currency, Cuisine, Longitude, Latitude, PhoneNumber, Url, WebsiteUrl and award

To prepare and clean this dataset, we will be creating additional columns to extract the data better. 

Firstly, getting the mean price of every resturant and using a currency converter to convert the mean price into USD -> this would facilitate better comparisons 

Secondly, getting the country from address, as the values in location are too specific. 

Lastly, we would also be dropping irrelevent columns such as Longitude, Latitude, PhoneNumber, Url and WebsiteUrl. As we are focusing on resturants that obtained at least 1 Michelin Star, we would also be dropping resturants that only obtained the Bib Gourmand Award.

<br>

**Currency Converter**

Download necessary library to convert currency, install, then import the library

The library can be downloaded from https://pypi.org/project/CurrencyConverter

In [1]:
pip install CurrencyConverter

Note: you may need to restart the kernel to use updated packages.


In [6]:
from currency_converter import CurrencyConverter

# sample test
c = CurrencyConverter()

c.convert(100, 'EUR', 'USD')

111.01

**Check for missing values before continuing**

Find missing values

In [7]:
michelindata.isnull().sum()

Name              0
Address           0
Location          0
MinPrice          1
MaxPrice          1
Currency          1
Cuisine           0
Longitude         0
Latitude          0
PhoneNumber     121
Url               0
WebsiteUrl     1127
Award             0
dtype: int64

- PhoneNumber and WebsiteUrl will not be tackled in this dataset as they are considered irrelevant in this      project so will only be tackling the values with no MinPrice and MaxPrice

In [8]:
michelindata[michelindata['MinPrice'].isnull()]

,Name,Address,Location,MinPrice,MaxPrice,Currency,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award
4968,Thai House,"254 Rose St., Danville, 94526, United States",Danville,NaN,NaN,NaN,Thai,-122.00018,37.82337,1.925820e+10,https://guide.michelin.com/en/california/danvi...,https://www.thethaihousedanville.com/,Bib Gourmand


MinPrice, MaxPrice will be 0 temporarily and Currency will be inserted USD as Address shows United States

In [9]:
michelindata.drop(4968, inplace=True)

In [10]:
michelindata[michelindata['MinPrice'].isnull()]

,Name,Address,Location,MinPrice,MaxPrice,Currency,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award


- Checking again for missing values in the MinPrice and MaxPrice column
- Missing values are successfully filled in

**Getting Mean Price and converting all the price to a standardised currency**

Convert data types of MinPrice and MaxPrice to integer

In [11]:
michelindata['MinPrice'] = michelindata['MinPrice'].str.replace(',', '').astype(int)
michelindata['MaxPrice'] = michelindata['MaxPrice'].str.replace(',', '').astype(int)

In [12]:
michelindata.dtypes

Name            object
Address         object
Location        object
MinPrice         int64
MaxPrice         int64
Currency        object
Cuisine         object
Longitude      float64
Latitude       float64
PhoneNumber    float64
Url             object
WebsiteUrl      object
Award           object
dtype: object

In [13]:
michelindata.iloc[0]['MinPrice']

225

 - Calculate Mean Price for each restaurant

In [14]:
michelindata['MeanPrice'] = (michelindata['MinPrice'] + michelindata['MaxPrice'])/2

In [15]:
michelindata.describe()

,MinPrice,MaxPrice,Longitude,Latitude,PhoneNumber,MeanPrice
count,6501.000000,6501.000000,6501.000000,6501.000000,6.380000e+03,6501.000000
mean,2229.932472,3767.321028,27.186057,40.946917,3.136822e+11,2998.626750
std,10138.560522,18104.793115,65.358048,11.919718,1.509684e+12,13896.793134
min,2.000000,2.000000,-123.719444,-23.638434,4.977184e+07,2.000000
25%,39.000000,61.000000,0.908524,35.689601,3.314430e+10,50.000000
50%,70.000000,120.000000,7.445899,43.610370,3.862600e+10,95.000000
75%,205.000000,300.000000,100.470749,48.818285,8.532864e+10,250.000000
max,200000.000000,380000.000000,139.847604,64.144365,4.981419e+13,290000.000000


 - Replace unsupported currencies with similar value currency for easier currency conversion

In [16]:
michelindata['Currency'].replace({'MOP':'HKD', 'RSD':'JPY', 'TWD':'THB'}, inplace=True)

In [17]:
michelindata['USDPrice'] = michelindata.apply(lambda row: c.convert(row['MeanPrice'], row['Currency'], 'USD'), axis=1)

In [18]:
michelindata['USDPrice']

0       249.772500
1       255.323000
2       320.818900
3       277.525000
4       245.887150
           ...    
6497     42.942984
6498     55.468021
6499     16.222905
6500     56.064452
6501     37.217253
Name: USDPrice, Length: 6501, dtype: float64

In [19]:
michelindata.describe()

,MinPrice,MaxPrice,Longitude,Latitude,PhoneNumber,MeanPrice,USDPrice
count,6501.000000,6501.000000,6501.000000,6501.000000,6.380000e+03,6501.000000,6501.000000
mean,2229.932472,3767.321028,27.186057,40.946917,3.136822e+11,2998.626750,93.959679
std,10138.560522,18104.793115,65.358048,11.919718,1.509684e+12,13896.793134,71.445504
min,2.000000,2.000000,-123.719444,-23.638434,4.977184e+07,2.000000,0.421051
25%,39.000000,61.000000,0.908524,35.689601,3.314430e+10,50.000000,46.069150
50%,70.000000,120.000000,7.445899,43.610370,3.862600e+10,95.000000,72.156500
75%,205.000000,300.000000,100.470749,48.818285,8.532864e+10,250.000000,126.504042
max,200000.000000,380000.000000,139.847604,64.144365,4.981419e+13,290000.000000,1016.076644


**Add Country data from Address**

In [20]:
michelindata['Country'] = michelindata['Address'].apply(lambda x: x.split(',')[-1])

Finally, drop irrelevant columns

In [26]:
michelindata = michelindata.drop(columns=['Longitude', 'Latitude', 'PhoneNumber', 'Url', 'WebsiteUrl'])

def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

michelindata = filter_rows_by_values(michelindata, "Award", ["Bib Gourmand"])

In [29]:
michelindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3215 entries, 0 to 3214
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       3215 non-null   object 
 1   Address    3215 non-null   object 
 2   Location   3215 non-null   object 
 3   MinPrice   3215 non-null   int64  
 4   MaxPrice   3215 non-null   int64  
 5   Currency   3215 non-null   object 
 6   Cuisine    3215 non-null   object 
 7   Award      3215 non-null   object 
 8   MeanPrice  3215 non-null   float64
 9   USDPrice   3215 non-null   float64
 10  Country    3215 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 301.4+ KB


**Save the data into another CSV file**

In [ ]:
michelindata.to_csv("CLEANED.csv")

#### Conclusion

We are left with a cleaned dataset with 3215 rows and 11 columns and with no NULL values. The columns are Name, Address, Location, MinPrice, MaxPrice, Currency, Cuisine, Award, MeanPrice, USDPrice and Country

We will mainly be using Country, Cuisine, USDPrice and Award in our data visualization and machine learning models.